In [24]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
df = pd.read_csv(r"E:\NITD\ML_Project\xtract-api\DataSet\arxiv_processed.csv")
df.head()

C:\Users\Hirakjyoti\AppData\Local\Temp\ipykernel_22032\3558869591.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"E:\NITD\ML_Project\xtract-api\DataSet\arxiv_processed.csv")


,id,title,abstract,authors,category_code,update_date,clean_title,clean_abstract,category
0,704.0033,Convergence of the discrete dipole approximati...,We performed a rigorous theoretical converge...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...",physics.optics physics.comp-ph,2022-03-31,convergence of the discrete dipole approximati...,we performed a rigorous theoretical convergenc...,"Physics – Computational Physics, Physics – Optics"
1,704.0045,Evolution of solitary waves and undular bores ...,This paper considers the propagation of shal...,"G.A. El, R.H.J. Grimshaw, A.M. Kamchatnov",nlin.PS nlin.SI,2007-09-23,evolution of solitary waves and undular bores ...,this paper considers the propagation of shallo...,Nonlinear Sciences – Exactly Solvable and Inte...
2,704.0048,Inference on white dwarf binary systems using ...,We report on the analysis of selected single...,"Alexander Stroeer, John Veitch, Christian Roev...",gr-qc astro-ph,2008-11-26,inference on white dwarf binary systems using ...,we report on the analysis of selected single s...,"Astrophysics, General Relativity and Quantum C..."
3,704.0106,Multiple Parton Scattering in Nuclei: Quark-qu...,Modifications to quark and antiquark fragmen...,"Andreas Schafer, Xin-Nian Wang and Ben-Wei Zhang",hep-ph nucl-th,2008-11-26,multiple parton scattering in nuclei quark qua...,modifications to quark and antiquark fragmenta...,"High Energy Physics – Phenomenology, Nuclear P..."
4,704.0148,Reexamination of spin decoherence in semicondu...,The longitudinal and transversal spin decohe...,"J. H. Jiang, Y. Y. Wang, and M. W. Wu",cond-mat.mtrl-sci,2008-01-20,reexamination of spin decoherence in semicondu...,the longitudinal and transversal spin decohere...,Condensed Matter – Materials Science


In [12]:
df['text'] = df['clean_title'] + " " + df['clean_abstract']
df.head()

,id,title,abstract,authors,category_code,update_date,clean_title,clean_abstract,category,text
0,704.0033,Convergence of the discrete dipole approximati...,We performed a rigorous theoretical converge...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...",physics.optics physics.comp-ph,2022-03-31,convergence of the discrete dipole approximati...,we performed a rigorous theoretical convergenc...,"Physics – Computational Physics, Physics – Optics",convergence of the discrete dipole approximati...
1,704.0045,Evolution of solitary waves and undular bores ...,This paper considers the propagation of shal...,"G.A. El, R.H.J. Grimshaw, A.M. Kamchatnov",nlin.PS nlin.SI,2007-09-23,evolution of solitary waves and undular bores ...,this paper considers the propagation of shallo...,Nonlinear Sciences – Exactly Solvable and Inte...,evolution of solitary waves and undular bores ...
2,704.0048,Inference on white dwarf binary systems using ...,We report on the analysis of selected single...,"Alexander Stroeer, John Veitch, Christian Roev...",gr-qc astro-ph,2008-11-26,inference on white dwarf binary systems using ...,we report on the analysis of selected single s...,"Astrophysics, General Relativity and Quantum C...",inference on white dwarf binary systems using ...
3,704.0106,Multiple Parton Scattering in Nuclei: Quark-qu...,Modifications to quark and antiquark fragmen...,"Andreas Schafer, Xin-Nian Wang and Ben-Wei Zhang",hep-ph nucl-th,2008-11-26,multiple parton scattering in nuclei quark qua...,modifications to quark and antiquark fragmenta...,"High Energy Physics – Phenomenology, Nuclear P...",multiple parton scattering in nuclei quark qua...
4,704.0148,Reexamination of spin decoherence in semicondu...,The longitudinal and transversal spin decohe...,"J. H. Jiang, Y. Y. Wang, and M. W. Wu",cond-mat.mtrl-sci,2008-01-20,reexamination of spin decoherence in semicondu...,the longitudinal and transversal spin decohere...,Condensed Matter – Materials Science,reexamination of spin decoherence in semicondu...


In [17]:
texts = df['text'].astype(str).tolist()

### tokenizing and padding the dataset

In [16]:
# extra step
df['len'] = df['text'].apply(lambda x: len(str(x).split()))
print(df['len'].describe())

# 95th percentile
max_len = int(np.percentile(df['len'], 95))
print(f"\n✅ Recommended max_len (95th percentile): {max_len}")

# ---------------------------------
# 2️⃣ Find ideal max_words
# ---------------------------------
# Flatten all words safely
all_words = []
for text in df['text']:
    all_words.extend(str(text).split())

# Count word frequencies
word_freq = Counter(all_words)

# Sort by frequency
total_words = sum(word_freq.values())
sorted_freqs = np.array(sorted(word_freq.values(), reverse=True))
cumulative_coverage = np.cumsum(sorted_freqs) / total_words

# 95% coverage threshold
max_words = np.argmax(cumulative_coverage >= 0.95) + 1
print(f"✅ Recommended max_words (95% coverage): {max_words}")

count    194065.000000
mean        155.522413
std          65.416309
min           1.000000
25%         105.000000
50%         152.000000
75%         202.000000
max         552.000000
Name: len, dtype: float64

✅ Recommended max_len (95th percentile): 271
✅ Recommended max_words (95% coverage): 8551


In [13]:
max_words = 10000
max_len = 300

In [20]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_len, padding='post')

In [27]:
embedding_dim = 100

model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu')  # output embedding layer
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
# GENERATE EMBEDDINGS

embeddings = model.predict(X, verbose=1)

embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)


6065/6065 ━━━━━━━━━━━━━━━━━━━━ 57s 9ms/step


In [31]:
# RECOMMENDATION FUNCTION

def recommend_similar(paper_index, top_n=5):
    sim_scores = cosine_similarity([embeddings[paper_index]], embeddings)[0]
    similar_idx = np.argsort(sim_scores)[::-1][1:top_n+1]
    recs = df.iloc[similar_idx][['title', 'authors', 'category']]
    recs['similarity'] = sim_scores[similar_idx]
    return recs

In [35]:
paper_index = 50000
print("📄 Input Paper:")
print(df.loc[paper_index, 'title'])
print("\n🔍 Recommended Papers:")
print(recommend_similar(paper_index))

📄 Input Paper:
Fermi polaron-polaritons in charge-tunable atomically thin
  semiconductors

🔍 Recommended Papers:
                                                    title  \
23505   Phase lagging model of brain response to exter...   
4916    Theory of Viscosity of Confined Fluids in Smal...   
102389  Exact Backpropagation in Binary Weighted Netwo...   
183967  From brane dynamics to a Kac-Moody invariant f...   
78511   Optical follow-up study of 32 high-redshift ga...   

                                                  authors  \
23505   Karthik Seetharaman, Hamidreza Namazi, Vladimi...   
4916        B. Mirzayi, G.A. Mansoori and M. Vafaie-Sefti   
102389                                      Yaniv Shulman   
183967                   Francois Englert, Laurent Houart   
78511   Hannah Zohren (1), Tim Schrabback (1), Remco F...   

                                                 category  similarity  
23505        Quantitative Biology – Neurons and Cognition    0.996738  
4916    